In [1]:
import re
from datetime import datetime
from geoip import geolite2
from collections import Counter
import psycopg2
from psycopg2 import OperationalError

## Файл

Посмотрим на файл с логами

In [2]:
f = open('logs.txt')
f.readlines()

['shop_api      | 2018-08-01 00:01:35 [YQ4WUDJV] INFO: 121.165.118.201 https://all_to_the_bottom.com/\n',
 'shop_api      | 2018-08-01 00:01:47 [E522JAEQ] INFO: 121.165.118.201 https://all_to_the_bottom.com/fresh_fish/\n',
 'shop_api      | 2018-08-01 00:03:02 [BHWP66G8] INFO: 121.165.118.201 https://all_to_the_bottom.com/\n',
 'shop_api      | 2018-08-01 00:04:27 [C0182Y6W] INFO: 121.165.118.201 https://all_to_the_bottom.com/canned_food/\n',
 'shop_api      | 2018-08-01 00:05:55 [OJKTKNP6] INFO: 121.165.118.201 https://all_to_the_bottom.com/\n',
 'shop_api      | 2018-08-01 00:07:25 [GQY4K4H9] INFO: 121.165.118.201 https://all_to_the_bottom.com/semi_manufactures/\n',
 'shop_api      | 2018-08-01 00:07:54 [ENZOWBC9] INFO: 121.165.118.201 https://all_to_the_bottom.com/canned_food/\n',
 'shop_api      | 2018-08-01 00:08:58 [7OEXWZ78] INFO: 121.165.118.201 https://all_to_the_bottom.com/semi_manufactures/\n',
 'shop_api      | 2018-08-01 00:10:35 [7O3F31OB] INFO: 121.165.118.201 https://al

In [3]:
f.close()

Ознакомившись со списком логов, выделим доступную информацию:
- дата
- время
- IP-адрес

А так же полезные данные из URL:
- категории товаров
- наименования товаров
- user_id
- goods_id
- cart_id

Создадим шаблоны (через регулярные выражения) для поиска данной информации в строке

In [129]:
date_pattern = re.compile('\d+[-]\d+[-]\d+')

time_pattern = re.compile('\d+[:]\d+[:]\d+')

ip_pattern = re.compile('\d{1,3}[.]\d{1,3}[.]\d{1,3}[.]\d{1,3}')
ip_digit_pattern = re.compile('(\d{1,3})[.](\d{1,3})[.](\d{1,3})[.](\d{1,3})')

user_id_pattern = re.compile('user_id=(\d+)')

goods_id_pattern = re.compile('goods_id=(\d+)')

amount_pattern = re.compile('amount=(\d+)')

cart_id_pattern = re.compile('cart_id=(\d+)')
paid_cart_id_pattern = re.compile('success_pay_(\d+)')

category_pattern = re.compile('.com/(\w+[^0-9])/')

product_pattern = re.compile('.com/\w+/(\w+)/')

#### Для удобства обработки данных создадим список с выделенной из строк информацией

In [9]:
logs_list = list()
with open('logs.txt') as logfile:
    for line in logfile:
        date = date_pattern.findall(line)[0]
        time = time_pattern.findall(line)[0]
        ip = ip_pattern.findall(line)[0]
        #ip_geoinfo = geolite2.lookup(ip)
        category = category_pattern.findall(
            line)[0] if category_pattern.findall(line) else ''
        product = product_pattern.findall(
            line)[0] if product_pattern.findall(line) else ''
        goods_id = goods_id_pattern.findall(
            line)[0] if goods_id_pattern.findall(line) else ''
        amount = amount_pattern.findall(
            line)[0] if amount_pattern.findall(line) else ''
        cart_id = cart_id_pattern.findall(
            line)[0] if cart_id_pattern.findall(line) else ''
        user_id = user_id_pattern.findall(
            line)[0] if user_id_pattern.findall(line) else ''
        paid_cart_id = paid_cart_id_pattern.findall(
            line)[0] if paid_cart_id_pattern.findall(line) else ''
        logs_info = (
            ip,
            # type(ip_geoinfo),
            date,
            time,
            category,
            product,
            goods_id,
            amount,
            cart_id,
            user_id,
            paid_cart_id
        )
        logs_list.append(logs_info)
logs_list

[('121.165.118.201', '2018-08-01', '00:01:35', '', '', '', '', '', '', ''),
 ('121.165.118.201',
  '2018-08-01',
  '00:01:47',
  'fresh_fish',
  '',
  '',
  '',
  '',
  '',
  ''),
 ('121.165.118.201', '2018-08-01', '00:03:02', '', '', '', '', '', '', ''),
 ('121.165.118.201',
  '2018-08-01',
  '00:04:27',
  'canned_food',
  '',
  '',
  '',
  '',
  '',
  ''),
 ('121.165.118.201', '2018-08-01', '00:05:55', '', '', '', '', '', '', ''),
 ('121.165.118.201',
  '2018-08-01',
  '00:07:25',
  'semi_manufactures',
  '',
  '',
  '',
  '',
  '',
  ''),
 ('121.165.118.201',
  '2018-08-01',
  '00:07:54',
  'canned_food',
  '',
  '',
  '',
  '',
  '',
  ''),
 ('121.165.118.201',
  '2018-08-01',
  '00:08:58',
  'semi_manufactures',
  '',
  '',
  '',
  '',
  '',
  ''),
 ('121.165.118.201',
  '2018-08-01',
  '00:10:35',
  'fresh_fish',
  '',
  '',
  '',
  '',
  '',
  ''),
 ('121.165.118.201',
  '2018-08-01',
  '00:11:00',
  'semi_manufactures',
  '',
  '',
  '',
  '',
  '',
  ''),
 ('121.165.118.201', 

## Visitors

Для дальнейшей обработки информации о посетителях создадим список с информацией об IP и user_ID каждого посещения сайта

In [6]:
visitors_list = []

with open('logs.txt') as logfile:
    for line in logfile:
        # добавим провервку IP адреса на условие: числа в IP адресе в диапазоне [0-255]
        ip_digits = ip_digit_pattern.findall(line)[0]
        for digit in ip_digits:
            if int(digit) <= 255 and int(digit) >= 0:
                ip = ip_pattern.findall(line)[0]
        visitors_info = (
            ip,
            user_id_pattern.findall(line)[0] if user_id_pattern.findall(line) else ''
        )
        visitors_list.append(visitors_info)
visitors_list

[('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', ''),
 ('121.165.118.201', '81270149216'),
 ('121.165.118.201', ''),
 ('217.89.121.82', ''),
 ('152.107.98.15', ''),
 ('217.89.121.82', ''),
 ('237.206.96.214', ''),
 ('114.0.179.225', ''),
 ('114.0.179.225', ''),
 ('237.206.96.214', ''),
 ('132.241.17.218', ''),
 ('152.107.98.15', ''),
 ('217.89.121.82

Создадим множество, содержащее информацию по уникальным IP-адресам и соответствующему user_ID (если таковой имеется)

Если IP-адресу принадлежал определённый user_ID, то мы будем считать такую пару основной (пара данного IP с отсутствующим user_ID в таком случае не будет учитываться)

In [7]:
visitors_set = set()
users_ip_set = set()

# добавим посетителей сайта с имеющимся user_ID
for visitor in visitors_list:
    if len(visitor[1]) > 1:
        visitors_set.add(visitor)
        users_ip_set.add(visitor[0])

# теперь добавим оставшиеся IP-адреса с проверкой условия, что такого IP ещё нету в множестве
for visitor in visitors_list:
    if visitor[0] not in users_ip_set:
        visitors_set.add(visitor)

visitors = list(visitors_set)
visitors

[('92.15.127.12', ''),
 ('179.178.212.192', ''),
 ('100.25.236.243', '80745983323'),
 ('73.230.89.113', '82159397822'),
 ('70.197.124.186', ''),
 ('152.149.206.47', '83537637476'),
 ('247.161.7.107', '86586665940'),
 ('152.216.67.105', '80716102037'),
 ('143.128.39.203', '85704840330'),
 ('126.224.106.111', '83363527361'),
 ('67.199.57.124', ''),
 ('92.7.2.251', ''),
 ('96.70.198.3', '86670517800'),
 ('128.121.88.24', '80240834459'),
 ('65.191.218.164', ''),
 ('240.52.6.99', ''),
 ('137.241.185.136', '87095092330'),
 ('9.59.189.208', '87160464911'),
 ('72.212.182.168', '81810056651'),
 ('112.31.144.7', '84287883780'),
 ('217.138.231.114', ''),
 ('134.168.221.27', ''),
 ('105.232.19.102', '85295071081'),
 ('21.205.2.250', '88687189305'),
 ('88.60.41.61', '81608293043'),
 ('16.134.192.203', '81752831276'),
 ('179.179.95.167', '80481856418'),
 ('194.170.93.236', ''),
 ('79.33.56.168', '87668203615'),
 ('54.67.227.30', '84400211876'),
 ('165.176.232.246', ''),
 ('118.6.165.127', '866783924

## Categories

Для выделения категорий товаров воспользуемся ранее созданным списком logs_list

In [10]:
categories_set = set()

for line in logs_list:
    if len(line[3]) > 0:
        categories_set.add(line[3])

categories = list()
count = 0
for category in categories_set:
    count += 1
    category_info = (
        count,
        category
    )
    categories.append(category_info)

categories

[(1, 'frozen_fish'),
 (2, 'semi_manufactures'),
 (3, 'caviar'),
 (4, 'fresh_fish'),
 (5, 'canned_food')]

## Goods

In [11]:
goods_set = set()

#ip = logs_list[0][0]
for line in sorted(logs_list):
    # if line[0] != ip:
    #    ip = line[0]
    if len(line[4]) > 0:
        category = line[3]
        product = line[4]
    if len(line[5]) > 0:
        goods_id = int(line[5])
    else:
        continue
    if len(category) > 0 and len(product) > 0 and goods_id > 0:
        goods_info = (
            goods_id,
            product,
            category
            #        line[3], # category
            #        line[4], # product
            #        line[5], # goods_id
        )
        #goods_info_list = list(goods_info)
        goods_set.add(goods_info)

goods_list = list(sorted(goods_set))
goods_list

[(1, 'crucian', 'fresh_fish'),
 (2, 'tuna', 'fresh_fish'),
 (3, 'salmon', 'fresh_fish'),
 (4, 'codfish', 'fresh_fish'),
 (5, 'marlene', 'fresh_fish'),
 (6, 'herring', 'fresh_fish'),
 (7, 'smelt', 'fresh_fish'),
 (8, 'shark', 'frozen_fish'),
 (9, 'eel', 'frozen_fish'),
 (10, 'salmon', 'frozen_fish'),
 (11, 'peljad', 'frozen_fish'),
 (12, 'pike', 'frozen_fish'),
 (13, 'carp', 'frozen_fish'),
 (14, 'crab_meat', 'semi_manufactures'),
 (15, 'squid_rings', 'semi_manufactures'),
 (16, 'salmon_cutlet', 'semi_manufactures'),
 (17, 'soup_set', 'semi_manufactures'),
 (18, 'stuffing_eel', 'semi_manufactures'),
 (19, 'sprats', 'canned_food'),
 (20, 'midii', 'canned_food'),
 (21, 'pate_of_tuna', 'canned_food'),
 (22, 'black_caviar', 'caviar'),
 (23, 'red_caviar', 'caviar'),
 (24, 'squash_caviar', 'caviar')]

Заменим значения категории продукта на соответствующий ID категории (из списка categories)

In [12]:
goods = list()
for product in goods_list:
    product = list(product)
    for category in categories:
        if product[2] == category[1]:
            product[2] = category[0]
            goods.append(tuple(product))
goods

[(1, 'crucian', 4),
 (2, 'tuna', 4),
 (3, 'salmon', 4),
 (4, 'codfish', 4),
 (5, 'marlene', 4),
 (6, 'herring', 4),
 (7, 'smelt', 4),
 (8, 'shark', 1),
 (9, 'eel', 1),
 (10, 'salmon', 1),
 (11, 'peljad', 1),
 (12, 'pike', 1),
 (13, 'carp', 1),
 (14, 'crab_meat', 2),
 (15, 'squid_rings', 2),
 (16, 'salmon_cutlet', 2),
 (17, 'soup_set', 2),
 (18, 'stuffing_eel', 2),
 (19, 'sprats', 5),
 (20, 'midii', 5),
 (21, 'pate_of_tuna', 5),
 (22, 'black_caviar', 3),
 (23, 'red_caviar', 3),
 (24, 'squash_caviar', 3)]

## Actions

Для выделения информации о действиях используем ранее написанные паттерны

In [88]:
actions_list = list()
count = 0

with open('logs.txt') as logfile:
    for line in logfile:
        count += 1
        date = date_pattern.findall(line)[0]+' '+time_pattern.findall(line)[0]
        ip = ip_pattern.findall(line)[0]
# добавим значения действий на сайте для удобства работы и восприятия информации
        if 'goods_id' in line:
            action_type = 'add_to_cart'
            cart_id = cart_id_pattern.findall(line)[0]
        elif 'success_pay' in line:
            action_type = 'success_transaction'
            cart_id = paid_cart_id_pattern.findall(line)[0]
        elif 'user_id' in line:
            action_type = 'payment'
            cart_id = cart_id_pattern.findall(line)[0]
        else:
            action_type = 'visit'
            cart_id = None
        category = category_pattern.findall(
            line)[0] if category_pattern.findall(line) else None
        actions_info = (
            count,
            datetime.strptime(date, "%Y-%m-%d %H:%M:%S"),
            ip,
            action_type,
            cart_id,
            category
        )
        actions_list.append(actions_info)

actions_list

[(1,
  datetime.datetime(2018, 8, 1, 0, 1, 35),
  '121.165.118.201',
  'visit',
  None,
  None),
 (2,
  datetime.datetime(2018, 8, 1, 0, 1, 47),
  '121.165.118.201',
  'visit',
  None,
  'fresh_fish'),
 (3,
  datetime.datetime(2018, 8, 1, 0, 3, 2),
  '121.165.118.201',
  'visit',
  None,
  None),
 (4,
  datetime.datetime(2018, 8, 1, 0, 4, 27),
  '121.165.118.201',
  'visit',
  None,
  'canned_food'),
 (5,
  datetime.datetime(2018, 8, 1, 0, 5, 55),
  '121.165.118.201',
  'visit',
  None,
  None),
 (6,
  datetime.datetime(2018, 8, 1, 0, 7, 25),
  '121.165.118.201',
  'visit',
  None,
  'semi_manufactures'),
 (7,
  datetime.datetime(2018, 8, 1, 0, 7, 54),
  '121.165.118.201',
  'visit',
  None,
  'canned_food'),
 (8,
  datetime.datetime(2018, 8, 1, 0, 8, 58),
  '121.165.118.201',
  'visit',
  None,
  'semi_manufactures'),
 (9,
  datetime.datetime(2018, 8, 1, 0, 10, 35),
  '121.165.118.201',
  'visit',
  None,
  'fresh_fish'),
 (10,
  datetime.datetime(2018, 8, 1, 0, 11),
  '121.165.118.20

Заменим значения категории продукта на соответствующий ID категории (из списка categories)

In [91]:
actions = list()
for action in actions_list:
    action = list(action)
    if action[5] != None:
        for category in categories:
            if action[5] == category[1]:
                action[5] = category[0]
                actions.append(tuple(action))
    else:
        action[5] = None
        actions.append(tuple(action))

actions

[(1,
  datetime.datetime(2018, 8, 1, 0, 1, 35),
  '121.165.118.201',
  'visit',
  None,
  None),
 (2,
  datetime.datetime(2018, 8, 1, 0, 1, 47),
  '121.165.118.201',
  'visit',
  None,
  4),
 (3,
  datetime.datetime(2018, 8, 1, 0, 3, 2),
  '121.165.118.201',
  'visit',
  None,
  None),
 (4,
  datetime.datetime(2018, 8, 1, 0, 4, 27),
  '121.165.118.201',
  'visit',
  None,
  5),
 (5,
  datetime.datetime(2018, 8, 1, 0, 5, 55),
  '121.165.118.201',
  'visit',
  None,
  None),
 (6,
  datetime.datetime(2018, 8, 1, 0, 7, 25),
  '121.165.118.201',
  'visit',
  None,
  2),
 (7,
  datetime.datetime(2018, 8, 1, 0, 7, 54),
  '121.165.118.201',
  'visit',
  None,
  5),
 (8,
  datetime.datetime(2018, 8, 1, 0, 8, 58),
  '121.165.118.201',
  'visit',
  None,
  2),
 (9,
  datetime.datetime(2018, 8, 1, 0, 10, 35),
  '121.165.118.201',
  'visit',
  None,
  4),
 (10,
  datetime.datetime(2018, 8, 1, 0, 11),
  '121.165.118.201',
  'visit',
  None,
  2),
 (11,
  datetime.datetime(2018, 8, 1, 0, 12, 35),
  '

## Purchases

In [15]:
purchases = set()
#paid_carts = list()

# with open('logs.txt') as logfile:
#    for line in logfile:
#        if 'success_pay' in line:
#            paid_cart_id = paid_cart_id_pattern.findall(line)[0]
#            paid_carts.append(paid_cart_id)
#        else:
#            continue

#        line[0], # ip
#        line[7], # cart_id
#        line[8], # user_id
#        line[5], # goods_id
#        line[6]  # amount

ip = sorted(logs_list, reverse=True)[0][0]
count = 0
for line in sorted(logs_list, reverse=True):
    if line[0] != ip:
        ip = line[0]
    if len(line[8]) > 0 and line[0] == ip:
        user_id = line[8]
        cart_id = line[7]
    if len(line[5]) > 0 and line[7] == cart_id:
        count += 1
        goods_id = line[5]
        amount = line[6]
        purchase_info = (
            count,
            cart_id,
            user_id,
            int(goods_id),
            int(amount)
        )
        purchases.add(purchase_info)

purchases = list(purchases)
sorted(purchases)

# with open('logs.txt') as logfile:
#    for line in logfile:
#        #cart_id = cart_id_pattern.findall(line)[0] if cart_id_pattern.findall(line) else ''
#        if cart_id_pattern.findall(line) and goods_id_pattern.findall(line) and cart_id_pattern.findall(line)[0] in paid_carts:
#            cart_id = cart_id_pattern.findall(line)[0]
#            goods_id = goods_id_pattern.findall(line)[0]
#            amount = amount_pattern.findall(line)[0]
#        if user_id_pattern.findall(line) and cart_id_pattern.findall(line)[0] == cart_id:
#            user_id = user_id_pattern.findall(line)[0]
#        else:
#            continue
#        transactions_info = (
#            cart_id,
#            user_id,
#            goods_id,
#            amount
#        )
#        transactions.append(transactions_info)

[(1, '8709', '89403419845', 12, 2),
 (2, '9610', '88902504585', 5, 2),
 (3, '9610', '88902504585', 24, 1),
 (4, '9510', '84681780333', 8, 1),
 (5, '9673', '81444320408', 21, 3),
 (6, '9003', '80998379368', 24, 1),
 (7, '9003', '80998379368', 22, 1),
 (8, '9003', '80998379368', 17, 2),
 (9, '9285', '81790803295', 10, 3),
 (10, '9139', '88321606296', 22, 2),
 (11, '9753', '86670517800', 14, 2),
 (12, '9753', '86670517800', 15, 1),
 (13, '9230', '88660842722', 13, 1),
 (14, '9767', '86271800664', 4, 3),
 (15, '9637', '86271800664', 22, 5),
 (16, '9123', '86271800664', 12, 3),
 (17, '9123', '86271800664', 11, 1),
 (18, '9404', '88676497521', 6, 2),
 (19, '9377', '88104254434', 16, 2),
 (20, '9301', '88104254434', 15, 2),
 (21, '8879', '85844654039', 21, 1),
 (22, '9101', '81014229147', 23, 1),
 (23, '9086', '82223065693', 21, 2),
 (24, '9086', '82223065693', 13, 3),
 (25, '9341', '84661565254', 13, 1),
 (26, '9341', '84661565254', 21, 2),
 (27, '9341', '84661565254', 12, 2),
 (28, '9600', 

## Работа с БД (создание таблиц и загрузка информации)

In [20]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


### Подключение к БД

Напишем функцию подключения к нашей БД

In [61]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [82]:
connection = create_connection(
    "all_to_the_bottom", "user", "abc123", "127.0.0.1", "5432"
)

Connection to PostgreSQL DB successful


### Создадим необходимые таблицы в нашей БД

In [80]:
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [24]:
create_visitors_table = """
CREATE TABLE IF NOT EXISTS visitors (
  ip TEXT PRIMARY KEY NOT NULL,
  user_id TEXT
)
"""

execute_query(connection, create_visitors_table)

Query executed successfully


In [30]:
create_categories_table = """
CREATE TABLE IF NOT EXISTS categories (
  id SERIAL PRIMARY KEY,
  category_name TEXT
)
"""

execute_query(connection, create_categories_table)

Query executed successfully


In [66]:
create_goods_table = """
CREATE TABLE IF NOT EXISTS goods (
  goods_id SERIAL PRIMARY KEY,
  product_name TEXT,
  category_id INTEGER REFERENCES categories(id)
)
"""

execute_query(connection, create_goods_table)

Query executed successfully


In [183]:
create_actions_table = """
CREATE TABLE IF NOT EXISTS actions (
  id SERIAL PRIMARY KEY,
  time TIMESTAMP,
  ip TEXT REFERENCES visitors(ip),
  action_type TEXT,
  cart_id TEXT,
  category_id INTEGER REFERENCES categories(id)
)
"""

execute_query(connection, create_actions_table)

Query executed successfully


In [191]:
create_purchases_table = """
CREATE TABLE IF NOT EXISTS purchases (
  id SERIAL PRIMARY KEY,
  cart_id TEXT,
  user_id TEXT,
  goods_id INTEGER REFERENCES goods(goods_id),
  amount INTEGER
)
"""

execute_query(connection, create_purchases_table)

Query executed successfully


### Загружаем данные в БД

In [29]:
visitor_records = ", ".join(["%s"] * len(visitors))

insert_query = (
    f"INSERT INTO visitors (ip, user_id) VALUES {visitor_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, visitors)

In [49]:
category_records = ", ".join(["%s"] * len(categories))

insert_query = (
    f"INSERT INTO categories (id, category_name) VALUES {category_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, categories)

In [69]:
goods_records = ", ".join(["%s"] * len(goods))

insert_query = (
    f"INSERT INTO goods (goods_id, product_name, category_id) VALUES {goods_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, goods)

In [187]:
action_records = ", ".join(["%s"] * len(actions))

insert_query = (
    f"INSERT INTO actions (id, time, ip, action_type, cart_id, category_id) VALUES {action_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, actions)

In [84]:
update_post_description = """
UPDATE
  actions
SET
  cart_id = NULL
WHERE
  cart_id = ''
"""

execute_query(connection, update_post_description)

Query executed successfully


In [194]:
purchase_records = ", ".join(["%s"] * len(purchases))

insert_query = (
    f"INSERT INTO purchases (id, cart_id, user_id, goods_id, amount) VALUES {purchase_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, purchases)

## Программный модуль

Предварительно напишем функцию для выгрузки данных из БД

In [59]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except OperationalError as e:
        print(f"The error '{e}' occurred")

Создадим класс, предоставляющий информацию согласно поставленным задачам

Данный код можно записать в отдельный файл с расширением .py и подключать его в качестве модуля

In [123]:
class Analytic_Reports():
    
    # из-за того что местоположение определяется для каждого IP после запроса к БД, 
    # время ожидания ответа получается большое
    def report_1(self):
        query_1 = """
        SELECT
            a.ip
        FROM 
            actions a
        """
        table_1 = execute_read_query(connection, query_1)
    
        countries = list()
        for visitor in table_1:
            location = geolite2.lookup(visitor[0])
            countries.append(location.country if location != None else '')
            countries_count = dict(Counter(countries))
    
        return dict(sorted(countries_count.items(), key=lambda item: item[1], reverse=True))
    
    # из-за того что местоположение определяется для каждого IP после запроса к БД, 
    # время ожидания ответа получается большое
    def report_2(self):
        query_2 = """
        SELECT
            a.ip
        FROM 
            actions a
            LEFT JOIN categories c ON c.id = a.category_id 
        WHERE 
            c.category_name = 'fresh_fish'
        """
        table_2 = execute_read_query(connection, query_2)

        countries_2 = list()
        for visitor in table_2:
            location = geolite2.lookup(visitor[0])
            countries_2.append(location.country if location != None else '')
            countries_count_2 = dict(Counter(countries_2))

        return dict(sorted(countries_count_2.items(), key=lambda item: item[1], reverse=True))
    
    
    def report_3(self):
        query_3 = """
        SELECT
            date_part('hour', a.time),
            count(a.id)
        FROM 
            actions a
            LEFT JOIN categories c ON c.id = a.category_id 
        WHERE 
            c.category_name = 'frozen_fish'
        GROUP BY 1
        """
        table_3 = execute_read_query(connection, query_3)

        night_visits = 0
        morning_visits = 0
        daytime_visits = 0
        evening_visits = 0
        for line in table_3:
            if line[0] in [0,1,2,3,4,5]:
                night_visits =+ line[1]
            elif line[0] in [6,7,8,9,10,11]:
                morning_visits =+ line[1]
            elif line[0] in [12,13,14,15,16]:
                daytime_visits =+ line[1]
            else:
                evening_visits =+ line[1]
        
        answer = print('Просмотров ночью:', night_visits, '\n'
                       'Просмотров утром:', morning_visits, '\n'
                       'Просмотров днём:', daytime_visits, '\n'
                       'Просмотров вечером:', evening_visits)
        return answer
    
    
    def report_4(self):
        query_4 = """
        SELECT
            date_part('month', a.time),
            date_part('day', a.time),
            date_part('hour', a.time),
            count(a.id)
        FROM 
            actions a
        GROUP BY 1,2,3
        ORDER BY 4 desc
        """
        table_4 = execute_read_query(connection, query_4)
        
        for line in table_4:
            answer = print('Количество запросов на сайт:', int(line[3]), '( месяц:', int(line[0]),
                           'день:', int(line[1]), 'час:', int(line[2]), ')')
            answer
        #return print('Максимальное количество запросов на сайт (за час):', int(table_4[0][3]), 
        #             '( месяц:', int(table_4[0][0]), 'день:', int(table_4[0][1]), 'час:', int(table_4[0][2]), ')')
    
    def report_5(self):
        query_5 = """
        SELECT
            count(a.cart_id),
            c.category_name
        FROM
            (
            SELECT
                p.cart_id,
                count(p.goods_id)
            FROM 
                purchases p
                LEFT JOIN goods g ON g.goods_id = p.goods_id
                LEFT JOIN categories c ON c.id = g.category_id
            WHERE
                c.category_name = 'semi_manufactures'
            GROUP BY 1
            HAVING
                count(p.goods_id) > 1
            ) a
            INNER JOIN purchases p ON p.cart_id = a.cart_id
            LEFT JOIN goods g ON g.goods_id = p.goods_id
            LEFT JOIN categories c ON c.id = g.category_id
        GROUP BY 2
        HAVING
            c.category_name NOT IN ('semi_manufactures')
        ORDER BY 1 desc
        """
        table_5 = execute_read_query(connection, query_5)
        
        for line in table_5:
            print('Товары из категории', str.upper(line[1]), 'купили совместно c товарами из semi_manufactures:', line[0], 'раз')
    
    def report_6(self):
        query_6 = """
        SELECT
            distinct a.cart_id
        FROM
            actions a
        WHERE
            a.cart_id IS NOT NULL
            
        EXCEPT
        
        SELECT
            p.cart_id
        FROM
            purchases p
        """
        table_6 = execute_read_query(connection, query_6)

        return print('Количество неоплаченных корзин:', len(table_6))
    
    def report_7(self):
        query_7 = """
        SELECT
            p.user_id,
            count(distinct p.cart_id)
        FROM
            purchases p
        GROUP BY 1
        HAVING
            count(distinct p.cart_id) > 1
        """
        table_7 = execute_read_query(connection, query_7)
        
        print('Количество пользователей, совершивших повторные покупки:', len(table_7))

Пример вывода отчёта:

In [128]:
reports = Analytic_Reports()

reports.report_3()

Просмотров ночью: 161 
Просмотров утром: 131 
Просмотров днём: 132 
Просмотров вечером: 130
